In [8]:
__author__ = 'timl'

from lxml import html
import requests
import time
import pymongo
from datetime import datetime, timedelta

start_time = time.time()

# Connection to Mongo DB
try:
    conn = pymongo.MongoClient('localhost', 27017)
    print "Connected successfully!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e
    conn

# test connection
db = conn['test-database4']



officeName = ['Bellevue/Bel-Red', 'Bellingham', 'Bremerton', 'Kennewick', 'Lacey', 'Lynnwood',
    'Mount Vernon', 'Parkland', 'Puyallup', 'Renton', 'Seattle: Downtown',
    'Seattle: West Seattle', 'Smokey Point', 'Spokane', 'Tacoma', 'Union Gap', 'Vancouver-North', 'Wenatchee']

officeWebAddress = ['https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=45&oid=23',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=46&oid=8',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=66&oid=9',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=315&oid=49',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=333&oid=5',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=371&oid=20',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=431&oid=12',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=495&oid=6',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=529&oid=34',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=542&oid=29',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=583&oid=24',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=583&oid=30',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=735&oid=21',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=619&oid=70',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=638&oid=36',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=668&oid=51',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=677&oid=40',
                    'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=703&oid=63'
                    ]

officeNameAll = ['Hoquiam','Kelso','Moses Lake','Port Angeles']

officeWebAddressAll = ['https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=289&oid=38',
                      'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=312&oid=42',
                      'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=428&oid=57',
                      'https://fortress.wa.gov/dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=510&oid=14']


#gets new license times
def findNewTimes(tree):
    license = tree.xpath("string(//*[@id=\"ctl00_Main_waittime\"]/text()[2])")
    times = license.split(" ")
    hour = None
    print license
    print times
    if not license:
        print('Website error')
        return None
    else:
        if times[0].isdigit():
            hour = float(times[0])
        if times[2].isdigit():
            minute = float(times[2])
        if hour != None:
            time = hour * 60 + minute
        else:
            time = minute
        return time


#gets renewal times, current time
def findRenewTimes(tree):
    renew = tree.xpath("string(//*[@id=\"ctl00_Main_waittime\"])")
    times = renew.split(" ")

    if not renew:
        print('Website error')
        return None
    else:
        hour = times[17][18:]
        if hour.isdigit():
            realHour = float(times[17][18:])
        else:
            realHour = '0'

        realMinute = float(times[19])

        if realHour == '0':
            time = realMinute
        elif realHour == '1':
            time = 60 + realMinute
        elif realHour == '2':
            time = 120 + realMinute
        else:
            time = realMinute

        return time


def go(name, address):
    try:
        page = requests.get(address)
        tree = html.fromstring(page.text)

    except:
        print("http connection failed, will try in 60 seconds")
        time.sleep(60)
        page = requests.get(address)
        print('{} ({})'.format(page.url, page.status_code))


    newWait = findNewTimes(tree)
    renewWait = findRenewTimes(tree)

    print name

    newLicense = {
        "DateTime": datetime.utcnow() + timedelta(hours=-7),
        "Location": name,
        "Wait Time": newWait
    }
    print('new license data:')
    print newLicense
    print('')

    renewLicense = {
        "DateTime": datetime.utcnow() + timedelta(hours=-7),
        "Location": name,
        "Wait Time": renewWait
    }


    print('renew license data:')
    print renewLicense

    db.test.alldol.insert({
        "Location:": name,
        "New License Wait Time:": newWait,
        "License Renewal Wait Time:": renewWait,
        "Timestamp": datetime.utcnow() + timedelta(hours=-7)
    })
    print '________________________________'
    print('')

#general function that finds times of new license and renewal
def findTimes(nameList, webList):
    count = 0
    while count < len(nameList):
        name = nameList[count]
        address = webList[count]
        #do stuff
        go(name, address)
        count += 1

#special case for offices that have all services (different xpath)
def findTimesAll(nameList, webList):
    count = 0

    while count < len(nameList):
        name = nameList[count]
        address = webList[count]

        while True:
            try:
                page = requests.get(address)
                break
            except:
                print("https connection failed, sleeping for 60 seconds and trying again")
                time.sleep(2)



        tree = html.fromstring(page.text)

        print name
        print('All Services data:')

        allWait = findNewTimes(tree)

        newLicense = {
            "DateTime": datetime.utcnow() + timedelta(hours=-7),
            "Location": name,
            "Wait Time": allWait
        }

        print newLicense

        db.test.alldol.insert({
            "Location:": name,
            "All Services Wait Time:": allWait,
            "Timestamp": datetime.utcnow() + timedelta(hours=-7)
        })
        print '________________________________'
        print('')
        count += 1


min = datetime.now().replace(hour=8, minute=00, second=0, microsecond=0)
max = datetime.now().replace(hour=17, minute=00, second=0, microsecond=0)


while True:
    curr = datetime.utcnow() + timedelta(hours=-7)
    while min < curr < max:
        runTime = time.time()
        findTimes(officeName, officeWebAddress)
        findTimesAll(officeNameAll, officeWebAddressAll)
        print("Run Time:")
        print("--- %s seconds ---" % (time.time() - runTime))
        print('')
        print("--- %s seconds ---" % (time.time() - start_time))
        time.sleep(60 - (time.time() - runTime))
        curr = datetime.utcnow() + timedelta(hours=-7)
        print curr

    print('waiting...')
    print('Current Time:')
    print curr
    print("--- %s seconds ---" % (time.time() - start_time))
    time.sleep(60)
    min = datetime.now().replace(hour=8, minute=00, second=0, microsecond=0)
    max = datetime.now().replace(hour=17, minute=00, second=0, microsecond=0)







Connected successfully!!!
0 hour(s), 14 min(s)
['0', 'hour(s),', '14', 'min(s)']
Bellevue/Bel-Red
new license data:
{'Wait Time': 14.0, 'Location': 'Bellevue/Bel-Red', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 10, 762000)}

renew license data:
{'Wait Time': 11.0, 'Location': 'Bellevue/Bel-Red', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 10, 762000)}


C:\Users\Tim Landowski\Anaconda2\lib\site-packages\ipykernel\__main__.py:147: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


________________________________

0 hour(s), 14 min(s)
['0', 'hour(s),', '14', 'min(s)']
Bellingham
new license data:
{'Wait Time': 14.0, 'Location': 'Bellingham', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 11, 271000)}

renew license data:
{'Wait Time': 0.0, 'Location': 'Bellingham', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 11, 271000)}
________________________________

0 hour(s), 32 min(s)
['0', 'hour(s),', '32', 'min(s)']
Bremerton
new license data:
{'Wait Time': 32.0, 'Location': 'Bremerton', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 11, 591000)}

renew license data:
{'Wait Time': 13.0, 'Location': 'Bremerton', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 11, 591000)}
________________________________

0 hour(s), 5 min(s)
['0', 'hour(s),', '5', 'min(s)']
Kennewick
new license data:
{'Wait Time': 5.0, 'Location': 'Kennewick', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 11, 868000)}

renew license data:
{'Wait Time': 0.0, 'Location': 'Kennewick'

C:\Users\Tim Landowski\Anaconda2\lib\site-packages\ipykernel\__main__.py:198: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


Kelso
All Services data:
0 hour(s), 1 min
['0', 'hour(s),', '1', 'min']
{'Wait Time': 1.0, 'Location': 'Kelso', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 16, 99000)}
________________________________

Moses Lake
All Services data:
0 hour(s), 3 min(s)
['0', 'hour(s),', '3', 'min(s)']
{'Wait Time': 3.0, 'Location': 'Moses Lake', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 16, 411000)}
________________________________

Port Angeles
All Services data:
0 hour(s), 5 min(s)
['0', 'hour(s),', '5', 'min(s)']
{'Wait Time': 5.0, 'Location': 'Port Angeles', 'DateTime': datetime.datetime(2016, 5, 13, 13, 48, 16, 705000)}
________________________________

Run Time:
--- 6.21599984169 seconds ---

--- 6.22000002861 seconds ---
2016-05-13 13:49:10.492000
0 hour(s), 12 min(s)
['0', 'hour(s),', '12', 'min(s)']
Bellevue/Bel-Red
new license data:
{'Wait Time': 12.0, 'Location': 'Bellevue/Bel-Red', 'DateTime': datetime.datetime(2016, 5, 13, 13, 49, 10, 752000)}

renew license data:
{'Wait Tim

ConnectionError: HTTPSConnectionPool(host='fortress.wa.gov', port=443): Max retries exceeded with url: /dol/dolprod/dsdoffices/OfficeInfo.aspx?cid=45&oid=23 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000000000295D470>: Failed to establish a new connection: [Errno 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))